## 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [2]:
import matplotlib.pyplot as plt
import missingno as msno

## 병합된 파일 불러오기

In [3]:
licensing_csv = pd.read_csv('/home/kmu/license/licensing.csv')



/tmp/ipykernel_30934/3411715468.py:1: DtypeWarning: Columns (2,3,6,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  licensing_csv = pd.read_csv('/home/kmu/license/licensing.csv')


In [4]:
licensing_csv.head()
licensing_csv.shape
licensing_csv.columns



#
#licensing_csv['개방자치단체코드'].isnull().sum()

#codes = licensing_csv['개방자치단체코드'].unique()
#codes.shape
#print(codes)


Index(['개방서비스명', '개방자치단체코드', '관리번호', '인허가일자', '영업상태구분코드', '영업상태명', '폐업일자',
       '휴업시작일자', '휴업종료일자', '재개업일자', '소재지전체주소', '도로명전체주소', '사업장명', '업태구분명',
       '좌표정보(X)', '좌표정보(Y)'],
      dtype='object')

In [5]:
licensing_csv['영업상태구분코드'].unique()
licensing_csv['영업상태명'].unique()

array(['폐업', '영업/정상', '휴업', '취소/말소/만료/정지/중지'], dtype=object)

In [6]:
filtered_df = licensing_csv.loc[licensing_csv['영업상태구분코드'].isin([1, 3]) ]

In [7]:
filtered_df.shape

(8920209, 16)

In [8]:
filtered_df = filtered_df[['개방자치단체코드', '인허가일자', '폐업일자', '영업상태명', '업태구분명']]
filtered_df.head(5)

,개방자치단체코드,인허가일자,폐업일자,영업상태명,업태구분명
0,3000000,20060803,20120217.0,폐업,치과병원
1,3000000,19820924,20130108.0,폐업,병원
2,3000000,20210624,NaN,영업/정상,한방병원
3,3000000,19770401,NaN,영업/정상,종합병원
4,3000000,19790324,NaN,영업/정상,종합병원


In [9]:
filtered_df['업태구분명'].unique()

array(['치과병원', '병원', '한방병원', ..., '공동탕업', '공동탕업+찜질시설서비스영업', '찜질시설서비스영업'],
      dtype=object)

In [10]:
filtered_df['업태구분명'].value_counts()

한식                                                                                 931825
즉석판매제조가공업                                                                          571305
기타                                                                                 496481
의류/패션/잡화/뷰티                                                                        381306
종합몰                                                                                278717
                                                                                    ...  
종합몰 교육/도서/완구/오락 가전 레져/여행/공연 건강/식품 컴퓨터/사무용품 기타 가구/수납용품 의류/패션/잡화/뷰티 자동차/자동차용품 상품권         1
의류/패션/잡화/뷰티 건강/식품 교육/도서/완구/오락 레져/여행/공연                                                  1
교육/도서/완구/오락 의류/패션/잡화/뷰티 가전 컴퓨터/사무용품 가구/수납용품 건강/식품 레져/여행/공연 자동차/자동차용품                    1
교육/도서/완구/오락 가전 컴퓨터/사무용품 의류/패션/잡화/뷰티 종합몰 가구/수납용품                                         1
종합몰 교육/도서/완구/오락 컴퓨터/사무용품 가구/수납용품 의류/패션/잡화/뷰티 건강/식품 레져/여행/공연 자동차/자동차용품 상품권 기타            1
Name: 업태구분

In [11]:
filtered_df['폐업일자'] = filtered_df['폐업일자'].fillna(0)
filtered_df['인허가일자'] = filtered_df['인허가일자'].fillna(0)
filtered_df.head(5)

,개방자치단체코드,인허가일자,폐업일자,영업상태명,업태구분명
0,3000000,20060803,20120217.0,폐업,치과병원
1,3000000,19820924,20130108.0,폐업,병원
2,3000000,20210624,0,영업/정상,한방병원
3,3000000,19770401,0,영업/정상,종합병원
4,3000000,19790324,0,영업/정상,종합병원


In [12]:
filtered_df['폐업일자'] = filtered_df['폐업일자'].astype(str)
filtered_df['인허가일자'] = filtered_df['인허가일자'].astype(str)
filtered_df.head(5)

,개방자치단체코드,인허가일자,폐업일자,영업상태명,업태구분명
0,3000000,20060803,20120217.0,폐업,치과병원
1,3000000,19820924,20130108.0,폐업,병원
2,3000000,20210624,0,영업/정상,한방병원
3,3000000,19770401,0,영업/정상,종합병원
4,3000000,19790324,0,영업/정상,종합병원


In [13]:
filtered_df['폐업일자'] = filtered_df['폐업일자'].str.extract(r'(\d+)')
filtered_df['인허가일자'] = filtered_df['인허가일자'].str.extract(r'(\d+)')
filtered_df.head(5)

,개방자치단체코드,인허가일자,폐업일자,영업상태명,업태구분명
0,3000000,20060803,20120217,폐업,치과병원
1,3000000,19820924,20130108,폐업,병원
2,3000000,20210624,0,영업/정상,한방병원
3,3000000,19770401,0,영업/정상,종합병원
4,3000000,19790324,0,영업/정상,종합병원


In [14]:
filtered_df['인허가년도'] = filtered_df['인허가일자'].str[:4]
filtered_df['폐업년도'] = filtered_df['폐업일자'].str[:4]

filtered_df.head(5)

,개방자치단체코드,인허가일자,폐업일자,영업상태명,업태구분명,인허가년도,폐업년도
0,3000000,20060803,20120217,폐업,치과병원,2006,2012
1,3000000,19820924,20130108,폐업,병원,1982,2013
2,3000000,20210624,0,영업/정상,한방병원,2021,0
3,3000000,19770401,0,영업/정상,종합병원,1977,0
4,3000000,19790324,0,영업/정상,종합병원,1979,0


In [15]:
filtered_df['영업상태코드'] = filtered_df['영업상태명'].map( {'영업/정상': 1, '폐업': 0} ).astype(int)

filtered_df.head(5)

,개방자치단체코드,인허가일자,폐업일자,영업상태명,업태구분명,인허가년도,폐업년도,영업상태코드
0,3000000,20060803,20120217,폐업,치과병원,2006,2012,0
1,3000000,19820924,20130108,폐업,병원,1982,2013,0
2,3000000,20210624,0,영업/정상,한방병원,2021,0,1
3,3000000,19770401,0,영업/정상,종합병원,1977,0,1
4,3000000,19790324,0,영업/정상,종합병원,1979,0,1


In [16]:
filtered_df.drop('영업상태명', axis=1, inplace= True)
filtered_df.drop('업태구분명', axis=1, inplace= True)
filtered_df

,개방자치단체코드,인허가일자,폐업일자,인허가년도,폐업년도,영업상태코드
0,3000000,20060803,20120217,2006,2012,0
1,3000000,19820924,20130108,1982,2013,0
2,3000000,20210624,0,2021,0,1
3,3000000,19770401,0,1977,0,1
4,3000000,19790324,0,1979,0,1
...,...,...,...,...,...,...
9500257,6520000,20140220,0,2014,0,1
9500258,6520000,20141010,0,2014,0,1
9500259,6520000,20161114,0,2016,0,1
9500260,6520000,20171030,0,2017,0,1


In [17]:
filtered_df.drop('폐업일자', axis=1, inplace= True)
filtered_df.drop('폐업년도', axis=1, inplace= True)
filtered_df

,개방자치단체코드,인허가일자,인허가년도,영업상태코드
0,3000000,20060803,2006,0
1,3000000,19820924,1982,0
2,3000000,20210624,2021,1
3,3000000,19770401,1977,1
4,3000000,19790324,1979,1
...,...,...,...,...
9500257,6520000,20140220,2014,1
9500258,6520000,20141010,2014,1
9500259,6520000,20161114,2016,1
9500260,6520000,20171030,2017,1


### licensing_csv 보기

In [ ]:
# licensing_csv

## 원본 저장후 새로운 변수 생성

In [ ]:
# licensing = licensing_csv

## 결측치 0으로 변환

In [ ]:
# licensing['폐업일자'] = licensing['폐업일자'].fillna(0)
# licensing['휴업시작일자'] = licensing['휴업시작일자'].fillna(0)
# licensing['휴업종료일자'] = licensing['휴업종료일자'].fillna(0)
# licensing['재개업일자'] = licensing['재개업일자'].fillna(0)

In [ ]:
# licensing.head(5)

## 일자형 데이터 str로 변환

In [ ]:
# licensing['폐업일자'] = licensing['폐업일자'].astype(str)
# licensing['휴업시작일자'] = licensing['휴업시작일자'].astype(str)
# licensing['휴업종료일자'] = licensing['휴업종료일자'].astype(str)
# licensing['재개업일자'] = licensing['재개업일자'].astype(str)

### 머리

In [ ]:
# licensing.head(5)

### 꼬리

In [ ]:
# licensing.tail(5)

## 숫자만 추출

In [ ]:
# licensing['폐업일자'] = licensing['폐업일자'].str.extract(r'(\d+)')
# licensing['휴업시작일자'] = licensing['휴업시작일자'].str.extract(r'(\d+)')
# licensing['휴업종료일자'] = licensing['휴업종료일자'].str.extract(r'(\d+)')
# licensing['재개업일자'] = licensing['재개업일자'].str.extract(r'(\d+)')

### 머리

In [ ]:
# licensing.head(5)

### 꼬리

In [ ]:
# licensing.tail(5)

## 년도만 추출

In [ ]:
# licensing['폐업일자(년)'] = licensing['폐업일자'].str[:4]
# licensing['휴업시작일자(년)'] = licensing['휴업시작일자'].str[:4]
# licensing['휴업종료일자(년)'] = licensing['휴업종료일자'].str[:4]
# licensing['재개업일자(년)'] = licensing['재개업일자'].str[:4]

### 머리

In [ ]:
# licensing.head(5)

### 꼬리

In [ ]:
# licensing.tail(5)

## 영업상태구분코드별 확인

### 코드 1

In [ ]:
# licensing[licensing['영업상태구분코드']==1].head()

### 코드 2

In [ ]:
# licensing[licensing['영업상태구분코드']==2].head()

### 코드 3

In [ ]:
# licensing[licensing['영업상태구분코드']==3].head()

### 코드4

In [ ]:
# licensing[licensing['영업상태구분코드']==3].head()

## 영업상태구분코드 1,2는 정상 3,4는 폐업으로 상태구분 열 생성

In [ ]:
# 상태구분 = []
# for row in licensing['영업상태구분코드']:
#     if row > 2:
#         상태구분.append('폐업')
#     else :
#         상태구분.append('정상')

# licensing['상태구분'] = 상태구분

### 머리

In [ ]:
# licensing.head(5)

### 꼬리

In [ ]:
# licensing.tail(5)

## 도로명과 소재지명을 법정동코드로 변환

### 도로명주소 추출

In [ ]:
# ad1 = licensing['도로명전체주소'].str.split(' ', expand=True)

##### ad1(도로명주소 전체보기)

In [ ]:
# ad1

#### 추출된 도로명주소 csv로 저장

In [ ]:
# ad1.to_csv("ad1.csv")

### 도로명 주소 불러오기

In [ ]:
# ad1 = pd.read_csv('/home/kmu/license/ad1.csv', usecols=[1,2])

In [ ]:
# ad1

In [ ]:
# ad1.to_csv('ad1_1.csv')

In [ ]:
# ad1 = pd.read_csv('/home/kmu/license/ad1_1.csv')

In [ ]:
# ad1

### 소재지주소 추출

In [ ]:
# ad2 = licensing['소재지전체주소'].str.split(' ', expand=True)

##### ad2(소재지주소 전체보기)

In [ ]:
# ad2

#### 추출된 소재지주소 csv로 저장

In [ ]:
# ad2.to_csv("ad2.csv")

### 소재지 주소 불러오기

In [ ]:
# ad2 = pd.read_csv('/home/kmu/license/ad2.csv', usecols=[1,2])

In [ ]:
# ad2

In [ ]:
# ad2.to_csv('ad2_1.csv')

In [ ]:
# ad2 = pd.read_csv('/home/kmu/license/ad2_1.csv')

In [ ]:
# ad2

### 주소지 합치기


In [ ]:
# ad1['법정동명'] = ad1['0'] + " " + ad1['1']

In [ ]:
# ad1

In [ ]:
# ad2['법정동명'] = ad2['0'] + " " + ad2['1']

In [ ]:
# ad2

In [ ]:
# ad1=ad1.iloc[:, 3]

In [ ]:
# ad1

In [ ]:
# ad2=ad2.iloc[:, 3]

In [ ]:
# ad2

In [ ]:
# ad1.to_csv('ad1_2.csv')
# ad2.to_csv('ad2_2.csv')

In [ ]:
# codeno = pd.read_csv('법정동코드.csv', delimiter=';',encoding='cp949')

In [ ]:
# codeno

In [ ]:
# ad= pd.read_csv('ad.csv',  delimiter=';',encoding='cp949')

In [ ]:
# msno.matrix(ad)
# plt.show()

In [ ]:
# ad['ad1'].isnull().sum()

In [ ]:
# ad['ad2'].isnull().sum()

In [ ]:
# ad

In [ ]:
# ad['ad'] = ad['ad1'].apply(lambda x: ad['ad1'] if x == 'NaN' else ad['ad2'])

In [ ]:
# ad.loc[ad['ad2'].isnull(), 'ad2'] = ad['ad1']

In [ ]:
# ad

In [ ]:
# ad['ad1'].isnull().sum()

In [ ]:
# ad['ad2'].isnull().sum()

In [ ]:
# ad

In [ ]:
# msno.matrix(ad)
# plt.show()

In [ ]:
# ad=ad.join(codeno.set_index('법정동명')['법정동코드'], on='ad2')

In [ ]:
## ad.set_index('ad2', inplace=True)
## ad['법정동코드'] = codeno['상장시가총액']

In [ ]:
# ad['법정동명'] = ad['ad2']
# ad

In [ ]:
# ad = pd.merge(ad, codeno, how='left', on='법정동명')
# ad 

In [ ]:
# ad

In [ ]:
# ad['ad2'].dropna()

In [ ]:
# ad['법정동코드'] = ad['법정동코드'].astype(int)

## 모델링

In [ ]:
df_x_data = filtered_df.drop(['영업상태코드'],axis=1)
df_y_data = filtered_df['영업상태코드']

df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_x_data, df_y_data, test_size=0.3, random_state=1206, stratify=df_y_data)

In [ ]:
print(df_x_train)

In [ ]:
print(df_x_test)

In [ ]:
print(df_y_train)

In [ ]:
print(df_y_test)

In [ ]:
from ctypes import wstring_at
from sklearn import model_selection
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

x_train, x_val, y_train, y_val = model_selection.train_test_split(df_x_train, df_y_train)

estimator = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
estimator.fit(x_train, y_train)

y_predict = estimator.predict(x_val)
score = metrics.r2_score(y_val, y_predict)
print(score)